In [17]:
import pandas as pd
import os
import re
import numpy as np

In [18]:
all_dfs = []


ignore_words = ['Código', 'Ano/Período', 'Nome','Funcional','cpf','Titulacao','Titulação','Nome.1']


for file in os.listdir():
    
    if file.startswith('~$'):
        continue
    if file.endswith('.xlsx') or file.endswith('.xls'):
        
        file_path = os.path.abspath(file)
        
        xls = pd.ExcelFile(file_path)
        
        for sheet_name in xls.sheet_names:
           
            first_line = pd.read_excel(file_path, sheet_name=sheet_name, nrows=1)
            
            skiprows = 1 if first_line.applymap(np.isreal).any().any() else 0
            
            df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=skiprows)
            
            df = df[~df.apply(lambda row: row.astype(str).str.contains('|'.join(ignore_words)).any(), axis=1)]
            
            df['SheetName'] = sheet_name
            
            if not sheet_name.isnumeric():
                
                numbers_in_title = re.findall(r'\d+', file)
                if numbers_in_title:
                    df['NumberInTitle'] = numbers_in_title[0]
            
            all_dfs.append(df)

C:\Users\Unip\AppData\Local\Temp\ipykernel_68736\773606872.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  skiprows = 1 if first_line.applymap(np.isreal).any().any() else 0
C:\Users\Unip\AppData\Local\Temp\ipykernel_68736\773606872.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  skiprows = 1 if first_line.applymap(np.isreal).any().any() else 0
C:\Users\Unip\AppData\Local\Temp\ipykernel_68736\773606872.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  skiprows = 1 if first_line.applymap(np.isreal).any().any() else 0
C:\Users\Unip\AppData\Local\Temp\ipykernel_68736\773606872.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  skiprows = 1 if first_line.applymap(np.isreal).any().any() else 0
C:\Users\Unip\AppData\Local\Temp\ipykernel_68736\773606872.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.m

In [19]:

geral_df = pd.concat(all_dfs)

In [20]:
geral_df.head(3)

,Disciplina,Unnamed: 1,Coordenador/professor,Unnamed: 3,Unnamed: 4,Unnamed: 5,SheetName,NumberInTitle,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
1,64A3,2019/2,TARSO ROCHA LULA PEREIRA,14663-8,828.366.984-20,MESTRE,Planilha1,01760001072,NaN,NaN,NaN,NaN,NaN
2,91A9,2019/2,TARSO ROCHA LULA PEREIRA,14663-8,828.366.984-20,MESTRE,Planilha1,01760001072,NaN,NaN,NaN,NaN,NaN
3,J925,2019/2,ÉLIO PESSOA CAZUZA,81140-8,060.441.734-96,MESTRE,Planilha1,01760001072,NaN,NaN,NaN,NaN,NaN


In [21]:
rename_columns = {'Disciplina': 'cod_disc', 'Unnamed: 1': 'ano/per','Coordenador/professor':'Nome','Unnamed: 3':'Funcional','Unnamed: 4':'cpf','Unnamed: 5':'Titulacao'}

geral_df = geral_df.rename(columns=rename_columns)

In [22]:
geral_df.head()

,cod_disc,ano/per,Nome,Funcional,cpf,Titulacao,SheetName,NumberInTitle,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
1,64A3,2019/2,TARSO ROCHA LULA PEREIRA,14663-8,828.366.984-20,MESTRE,Planilha1,01760001072,NaN,NaN,NaN,NaN,NaN
2,91A9,2019/2,TARSO ROCHA LULA PEREIRA,14663-8,828.366.984-20,MESTRE,Planilha1,01760001072,NaN,NaN,NaN,NaN,NaN
3,J925,2019/2,ÉLIO PESSOA CAZUZA,81140-8,060.441.734-96,MESTRE,Planilha1,01760001072,NaN,NaN,NaN,NaN,NaN
4,J935,2019/2,TARSO ROCHA LULA PEREIRA,14663-8,828.366.984-20,MESTRE,Planilha1,01760001072,NaN,NaN,NaN,NaN,NaN
5,J945,2019/2,PAULO SÉRGIO SILVA S LACERDA,942537,010.259.094-02,ESPECIALISTA,Planilha1,01760001072,NaN,NaN,NaN,NaN,NaN


In [ ]:
DECLARE @NextKey INT;

SELECT @NextKey = MAX(Chave) + 1 FROM SuaTabela;

INSERT INTO SuaTabela (Chave, OutroCampo)
VALUES (@NextKey, 'NovoValor');

In [23]:
geral_df.to_excel('aquivo_final.xlsx',index = False)